In [48]:
import pandas as pd

### Load Raw

In [49]:
raw_df = pd.read_csv('./data/train.csv')
data_df = raw_df

### Rename Columns

In [50]:
data_df = data_df.rename(columns={
    "RoomService"   : "spend_room_service",
    "FoodCourt"     : "spend_food_court",
    "ShoppingMall"  : "spend_shopping_mall",
    "Spa"           : "spend_spa",
    "VRDeck"        : "spend_vr_deck",
    })

In [51]:
data_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,spend_room_service,spend_food_court,spend_shopping_mall,spend_spa,spend_vr_deck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


# Cleaning

## Break-ups

There are a few compound features that would be better served broken down:

**PassengerId** 


`gggg_pp` - group_num / num_within

**Cabin** 

`deck/num/side` - _P|S_ port|starboard




In [52]:
data_df['group_id'] = data_df['PassengerId'].apply(lambda x: x.split('_')[0])
data_df['group_num'] = data_df['PassengerId'].apply(lambda x: x.split('_')[1])

In [53]:
data_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,spend_room_service,spend_food_court,spend_shopping_mall,spend_spa,spend_vr_deck,Name,Transported,group_id,group_num
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0001,01
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,0002,01
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0003,01
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0003,02
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,0004,01


### Cabin is tricky

There are multiple values including nulls in cabin. Let's explore.

In [54]:
data_df.Cabin.value_counts()

G/734/S     8
G/109/P     7
B/201/P     7
G/1368/P    7
G/981/S     7
           ..
G/556/P     1
E/231/S     1
G/545/S     1
G/543/S     1
F/947/P     1
Name: Cabin, Length: 6560, dtype: int64

## LEARN: How to tell string values

Looking for string values in a feature should have an easier path.

In [55]:
def is_str(value):
    if value is None: return False
    # if value.isna(): return False
    return type(value) == str

cabin_is_string_ser = data_df.Cabin.apply(lambda x: is_str(x))
# to display 
cabin_is_string_ser.value_counts()

True     8494
False     199
Name: Cabin, dtype: int64

In [56]:
# data_df.where(cabin_is_string_ser == False)
data_df[cabin_is_string_ser == False]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,spend_room_service,spend_food_court,spend_shopping_mall,spend_spa,spend_vr_deck,Name,Transported,group_id,group_num
15,0012_01,Earth,False,NaN,TRAPPIST-1e,31.0,False,32.0,0.0,876.0,0.0,0.0,Justie Pooles,False,0012,01
93,0101_01,Mars,True,NaN,TRAPPIST-1e,31.0,False,0.0,0.0,0.0,0.0,0.0,Book Trad,True,0101,01
103,0110_01,Europa,False,NaN,TRAPPIST-1e,32.0,False,0.0,410.0,6.0,3929.0,764.0,Graviph Aloubtled,False,0110,01
222,0239_01,Mars,False,NaN,TRAPPIST-1e,37.0,False,637.0,0.0,0.0,92.0,319.0,Diedow Resty,False,0239,01
227,0244_01,Mars,True,NaN,TRAPPIST-1e,43.0,False,0.0,0.0,0.0,0.0,0.0,Froos Sad,True,0244,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8209,8772_02,Europa,False,NaN,55 Cancri e,53.0,False,0.0,1127.0,0.0,3939.0,NaN,Naosura Motled,False,8772,02
8475,9057_01,Europa,False,NaN,55 Cancri e,36.0,True,132.0,3479.0,0.0,3786.0,0.0,Coxan Statch,False,9057,01
8485,9069_03,Europa,True,NaN,55 Cancri e,25.0,False,0.0,0.0,0.0,0.0,0.0,Bath Brakeng,True,9069,03
8509,9081_03,Earth,True,NaN,TRAPPIST-1e,1.0,False,0.0,0.0,0.0,0.0,0.0,Beula Clemondsey,False,9081,03


In [57]:
data_df[data_df.Cabin.isna()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,spend_room_service,spend_food_court,spend_shopping_mall,spend_spa,spend_vr_deck,Name,Transported,group_id,group_num
15,0012_01,Earth,False,NaN,TRAPPIST-1e,31.0,False,32.0,0.0,876.0,0.0,0.0,Justie Pooles,False,0012,01
93,0101_01,Mars,True,NaN,TRAPPIST-1e,31.0,False,0.0,0.0,0.0,0.0,0.0,Book Trad,True,0101,01
103,0110_01,Europa,False,NaN,TRAPPIST-1e,32.0,False,0.0,410.0,6.0,3929.0,764.0,Graviph Aloubtled,False,0110,01
222,0239_01,Mars,False,NaN,TRAPPIST-1e,37.0,False,637.0,0.0,0.0,92.0,319.0,Diedow Resty,False,0239,01
227,0244_01,Mars,True,NaN,TRAPPIST-1e,43.0,False,0.0,0.0,0.0,0.0,0.0,Froos Sad,True,0244,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8209,8772_02,Europa,False,NaN,55 Cancri e,53.0,False,0.0,1127.0,0.0,3939.0,NaN,Naosura Motled,False,8772,02
8475,9057_01,Europa,False,NaN,55 Cancri e,36.0,True,132.0,3479.0,0.0,3786.0,0.0,Coxan Statch,False,9057,01
8485,9069_03,Europa,True,NaN,55 Cancri e,25.0,False,0.0,0.0,0.0,0.0,0.0,Bath Brakeng,True,9069,03
8509,9081_03,Earth,True,NaN,TRAPPIST-1e,1.0,False,0.0,0.0,0.0,0.0,0.0,Beula Clemondsey,False,9081,03


In [58]:
def cabin_parts(cabin):
    if cabin is None: return None
    if type(cabin) != str: return None
    return cabin.split("/")
def get_part(cabin, index):
    parts = cabin_parts(cabin)
    if parts is None: return None
    if len(parts) != 3: return None
    return parts[index]

data_df["cabin_deck"] = data_df.Cabin.apply(lambda x: get_part(x, 0))
data_df["cabin_num"] = data_df.Cabin.apply(lambda x: get_part(x, 1))
data_df["cabin_side"] = data_df.Cabin.apply(lambda x: get_part(x, 2))

## Nulls

In [59]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PassengerId          8693 non-null   object 
 1   HomePlanet           8492 non-null   object 
 2   CryoSleep            8476 non-null   object 
 3   Cabin                8494 non-null   object 
 4   Destination          8511 non-null   object 
 5   Age                  8514 non-null   float64
 6   VIP                  8490 non-null   object 
 7   spend_room_service   8512 non-null   float64
 8   spend_food_court     8510 non-null   float64
 9   spend_shopping_mall  8485 non-null   float64
 10  spend_spa            8510 non-null   float64
 11  spend_vr_deck        8505 non-null   float64
 12  Name                 8493 non-null   object 
 13  Transported          8693 non-null   bool   
 14  group_id             8693 non-null   object 
 15  group_num            8693 non-null   o

In [69]:
data_df[data_df['Cabin'].isna()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,spend_room_service,spend_food_court,spend_shopping_mall,...,Name,Transported,group_id,group_num,cabin_deck,cabin_num,cabin_side,spend_total,spend_high_total,spend_low_total
15,0012_01,Earth,False,NaN,TRAPPIST-1e,31.0,False,32.0,0.0,876.0,...,Justie Pooles,False,0012,01,None,None,None,908.0,32.0,876.0
93,0101_01,Mars,True,NaN,TRAPPIST-1e,31.0,False,0.0,0.0,0.0,...,Book Trad,True,0101,01,None,None,None,0.0,0.0,0.0
103,0110_01,Europa,False,NaN,TRAPPIST-1e,32.0,False,0.0,410.0,6.0,...,Graviph Aloubtled,False,0110,01,None,None,None,5109.0,4693.0,416.0
222,0239_01,Mars,False,NaN,TRAPPIST-1e,37.0,False,637.0,0.0,0.0,...,Diedow Resty,False,0239,01,None,None,None,1048.0,1048.0,0.0
227,0244_01,Mars,True,NaN,TRAPPIST-1e,43.0,False,0.0,0.0,0.0,...,Froos Sad,True,0244,01,None,None,None,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8209,8772_02,Europa,False,NaN,55 Cancri e,53.0,False,0.0,1127.0,0.0,...,Naosura Motled,False,8772,02,None,None,None,5066.0,3939.0,1127.0
8475,9057_01,Europa,False,NaN,55 Cancri e,36.0,True,132.0,3479.0,0.0,...,Coxan Statch,False,9057,01,None,None,None,7397.0,3918.0,3479.0
8485,9069_03,Europa,True,NaN,55 Cancri e,25.0,False,0.0,0.0,0.0,...,Bath Brakeng,True,9069,03,None,None,None,0.0,0.0,0.0
8509,9081_03,Earth,True,NaN,TRAPPIST-1e,1.0,False,0.0,0.0,0.0,...,Beula Clemondsey,False,9081,03,None,None,None,0.0,0.0,0.0


In [67]:
spend_cols = [
    "spend_room_service",
    "spend_food_court",
    "spend_shopping_mall",
    "spend_spa",
    "spend_vr_deck",
    ]
tdf = data_df
tdf[spend_cols] = data_df[spend_cols].fillna(0.0)
tdf.isna().sum().sort_values(ascending=False)

CryoSleep              217
VIP                    203
HomePlanet             201
Name                   200
Cabin                  199
cabin_side             199
cabin_num              199
cabin_deck             199
Destination            182
Age                    179
PassengerId              0
group_num                0
spend_high_total         0
spend_total              0
spend_vr_deck            0
group_id                 0
Transported              0
spend_spa                0
spend_shopping_mall      0
spend_food_court         0
spend_room_service       0
spend_low_total          0
dtype: int64

In [61]:
data_df.isna().sum().sort_values(ascending=False)

CryoSleep              217
spend_shopping_mall    208
VIP                    203
HomePlanet             201
Name                   200
cabin_num              199
cabin_deck             199
cabin_side             199
Cabin                  199
spend_vr_deck          188
spend_food_court       183
spend_spa              183
Destination            182
spend_room_service     181
Age                    179
Transported              0
group_id                 0
group_num                0
PassengerId              0
dtype: int64

In [62]:
data_df[data_df['CryoSleep'].isna()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,spend_room_service,spend_food_court,spend_shopping_mall,spend_spa,spend_vr_deck,Name,Transported,group_id,group_num,cabin_deck,cabin_num,cabin_side
92,0099_02,Earth,NaN,G/12/P,TRAPPIST-1e,2.0,False,0.0,0.0,0.0,0.0,0.0,Thewis Connelson,True,0099,02,G,12,P
98,0105_01,Earth,NaN,F/21/P,TRAPPIST-1e,27.0,False,0.0,0.0,570.0,2.0,131.0,Carry Cleachrand,False,0105,01,F,21,P
104,0110_02,Europa,NaN,B/5/P,TRAPPIST-1e,40.0,False,0.0,331.0,0.0,0.0,1687.0,Aldeba Bootious,False,0110,02,B,5,P
111,0115_01,Mars,NaN,F/24/P,TRAPPIST-1e,26.0,False,0.0,0.0,0.0,0.0,NaN,Rohs Pead,True,0115,01,F,24,P
152,0173_01,Earth,NaN,E/11/S,TRAPPIST-1e,58.0,False,0.0,985.0,0.0,5.0,0.0,Hilip Grifford,True,0173,01,E,11,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8620,9197_01,Europa,NaN,C/308/P,55 Cancri e,44.0,False,0.0,0.0,0.0,0.0,0.0,Bellus Platch,True,9197,01,C,308,P
8651,9227_05,Earth,NaN,G/1498/P,TRAPPIST-1e,8.0,False,0.0,0.0,0.0,0.0,0.0,Hard Hinglendez,False,9227,05,G,1498,P
8664,9246_01,Earth,NaN,G/1490/S,TRAPPIST-1e,32.0,False,0.0,0.0,0.0,0.0,0.0,NaN,True,9246,01,G,1490,S
8675,9259_01,Earth,NaN,F/1893/P,TRAPPIST-1e,44.0,False,1030.0,1015.0,0.0,11.0,NaN,Annah Gilleyons,True,9259,01,F,1893,P


## Spending Totals

Let's combine some spending to see if there are "types" of spending that correlate

In [63]:
spend_cols = [
    "spend_room_service",
    "spend_food_court",
    "spend_shopping_mall",
    "spend_spa",
    "spend_vr_deck",
    ]
data_df["spend_total"] = data_df[spend_cols].sum(axis=1)

spend_high_cols = [
    "spend_room_service",
    "spend_spa",
    "spend_vr_deck"]
data_df["spend_high_total"] = data_df[spend_high_cols].sum(axis=1)

spend_low_cols = [
    "spend_food_court",
    "spend_shopping_mall",
    ]
data_df["spend_low_total"] = data_df[spend_low_cols].sum(axis=1)

In [64]:
data_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,spend_room_service,spend_food_court,spend_shopping_mall,...,Name,Transported,group_id,group_num,cabin_deck,cabin_num,cabin_side,spend_total,spend_high_total,spend_low_total
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,...,Maham Ofracculy,False,0001,01,B,0,P,0.0,0.0,0.0
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,...,Juanna Vines,True,0002,01,F,0,S,736.0,702.0,34.0
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,...,Altark Susent,False,0003,01,A,0,S,10383.0,6807.0,3576.0
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,...,Solam Susent,False,0003,02,A,0,S,5176.0,3522.0,1654.0
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,...,Willy Santantines,True,0004,01,F,1,S,1091.0,870.0,221.0
